In [56]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import itertools as it
import tabulate as tb
from functools import reduce
from operator import concat
from collections import OrderedDict
import math

DEFAULT_BLOCKS = 12
DEFAULT_NODES = 3
DEFAULT_K = 2
DEFAULT_M = 1
DEFAULT_LAYOUT = "Inline"
DEFAULT_DIRECTION = "Horizontal"


def roundBlocks(factor, ob):
    return (factor - (ob % factor)) if (ob % factor) > 0 else 0


def makeEcGroups(
        blocks,
        k, m,
        direction,
        offset=0):

    offset = len(blocks) if offset == 0 else offset
    res = []

    groups = (len(blocks) // k)
    steps = len(blocks) // groups
    if direction == "Horizontal":
        for b in range(0, len(blocks), steps):
            res.append(
                blocks[b:b + steps] +
                list(range(offset + 1, (offset + 1 + m))))
            offset += m
    else:
        for s in range(0, groups):
            res.append(
                blocks[s:s+groups+offset:groups] +
                list(range(offset + 1, (offset + 1 + m))))
            offset += m

    return res


def mapNodes(blocks, hosts, offset=0):
    width = getZeros(len(blocks) or BC.value)
    formatBlock = getBlockFormat(width)
    res = OrderedDict()

    for i, _ in enumerate(blocks):
        key = "node" + str((i % hosts)+1)
        res[key] = (key in res and res[key]) or []
        res[key].append(formatBlock(blocks[i]))

    return res


def getBlockFormat(width):
    return lambda b: "{b:0{fmt}}".format(b=b, fmt=width)


def getZeros(width):
    return int(math.log(width, 10) + 1)


def makeReadSeq(K, M, ecGroups, direction, layout):
    readSequence = []
    if layout == "Append":
        if direction == "Horizontal":
            readSequence = reduce(
                concat,
                [g[:K] for g in ecGroups] + [g[-M:] for g in ecGroups])
        else:
            readSequence = reduce(concat, zip(*ecGroups))
    else:
        readSequence = reduce(concat, ecGroups)

    return readSequence


class ECPlacement:
    blocks = []
    nodesMap = OrderedDict()
    blockSeq = []
    ecGroups = []
    rb = 0

    def __init__(
            self,
            k=DEFAULT_K, m=DEFAULT_M,
            layout=DEFAULT_LAYOUT,
            direction=DEFAULT_LAYOUT,
            nodes=DEFAULT_NODES,
            bc=DEFAULT_BLOCKS):
        self.blocks = list(range(1, bc + 1))
        self.k = k
        self.m = m
        self.layout = layout
        self.direction = direction
        self.nodes = nodes
        self.bc = bc
        self.blocks = []
        self.nodesMap = OrderedDict()
        self.blockSeq = []
        self.ecGroups = []
        self.rb = 0

    def reset(self):
        self.k = DEFAULT_K
        self.m = DEFAULT_M
        self.layout = DEFAULT_LAYOUT
        self.direction = DEFAULT_DIRECTION
        self.nodes = DEFAULT_NODES
        self.bc = DEFAULT_BLOCKS
        self.rb = 0

        self.blocks = list(range(1, self.bc + 1))
        self.nodesMap = OrderedDict()
        self.blockSeq = []
        self.ecGroups = []

    def update(self, k, m, bc, direction, layout, nodes, append):
        print(k, m, bc, direction, layout, nodes, append)

        if append:
            if bc > self.bc:
                appendBlocks = bc - self.bc
                appendBlocks = appendBlocks + roundBlocks(k, appendBlocks)
                self.blocks += list(
                    range(self.bc + 1, bc + appendBlocks + 1))

                if len(self.blocks) > 0:
                    blockSeqLen = len(self.blockSeq)
                    print(blockSeqLen)

                    newBlocks = list(
                        range(blockSeqLen + 1, blockSeqLen + appendBlocks + 1))

                    self.ecGroups += makeEcGroups(
                        newBlocks, k, m, direction, blockSeqLen + appendBlocks)  # Calculate EC groups

                    self.blockSeq = makeReadSeq(
                        k, m, self.ecGroups, direction, layout)

                    self.nodesMap = mapNodes(
                        self.blockSeq, nodes).items()
        else:
            # Calculate geometry
            self.rb = roundBlocks(k, bc)
            # Generate source blocks
            self.blocks = list(
                range(1, bc + 1 + self.rb))

            if len(self.blocks) > 0:
                self.ecGroups = makeEcGroups(
                    self.blocks, k, m, direction)      # Calculate EC groups

                self.blockSeq = makeReadSeq(
                    k, m, self.ecGroups, direction, layout)

                self.nodesMap = mapNodes(
                    self.blockSeq, nodes).items()

        self.bc = bc
        self.k = k
        self.m = m
        self.direction = dir
        self.layout = layout
        self.nodes = nodes

        self.render()

    def render(self):
        width = getZeros(len(self.blockSeq) or self.bc)
        formatBlock = getBlockFormat(width)

        print("\n")
        print("Rounded Blocks:", self.rb)
        print("Encoding Groups:", len(self.ecGroups))
        print("K + M = ", self.k + self.m)
        print("Source Blocks:", ", ".join(map(formatBlock, self.blocks)))
        print("Read Layout:", ", ".join(map(formatBlock, self.blockSeq)))

        groupsTable = tb.tabulate(
            [[", ".join(map(formatBlock, g[:self.k])), ", ".join(map(formatBlock, g[-self.m:]))]
                for g in self.ecGroups], ["Source", "Parity"])
        print("\n", groupsTable)

        nodesTable = tb.tabulate(
            list(map(list, self.nodesMap)), ["Nodes", "Blocks"])
        print("\n", nodesTable)


ecPlacement = ECPlacement()


def init(k, m, bc, dir, layout, nodes, append):
    # K      - The number of source blocks to encode
    # BC     - Initial block count
    # Dir    - Coding direction
    # Layout - Block layout
    # Nodes  - Nodes count
    ##

    ecPlacement.update(k, m, bc, dir, layout, nodes, append)


MAX_BLOCKS = 256
K = widgets.IntSlider(description="K", min=2, max=MAX_BLOCKS)
M = widgets.IntSlider(description="M", min=1, max=MAX_BLOCKS)
BC = widgets.IntSlider(description="Origina blocks count",
                       min=1, max=10000, value=DEFAULT_BLOCKS)

D = widgets.Dropdown(options=['Horizontal', 'Vertical'],
                     description="EC Coding Direction")

BlocksLayout = widgets.Dropdown(
    options=['Inline', 'Append'],
    description="Blocks Manifest Layout")

Clear = widgets.Button(description='Clear')
Nodes = widgets.IntSlider(
    description="Nodes", min=K.value + M.value, max=MAX_BLOCKS)

Append = widgets.Checkbox(
    value=False,
    description='Append',
    disabled=False,
    indent=False)

ui = widgets.VBox([
    widgets.Label(value="EC Layout and Placement"),
    widgets.VBox([K, M, BC, D, BlocksLayout, Nodes, Append]),
    Clear])


def clearModelClick(button):
    ecPlacement.reset()
    K.value = K.min
    M.value = M.min
    Nodes.value = Nodes.min = K.value + M.value
    BC.value = 12
    D.value = "Horizontal"
    BlocksLayout.value = "Inline"
    appendBlocks.value = 0
    appendNodes = 0
    Append.value = False


Clear.on_click(clearModelClick)


def updateKRange(*args):
    K.max = MAX_BLOCKS - M.value
    Nodes.value = K.value + M.value


M.observe(updateKRange, 'value')


def updateMRange(*args):
    M.max = MAX_BLOCKS - K.value
    Nodes.value = K.value + M.value


K.observe(updateMRange, 'value')


def updateNodesRange(*args):
    Nodes.min = K.value + M.value


Nodes.observe(updateNodesRange, 'value')

init = widgets.interactive_output(
    init,
    {
        'k': K,
        'm': M,
        'bc': BC,
        'dir': D,
        'layout': BlocksLayout,
        'nodes': Nodes,
        'append': Append})

display(ui, init)


Output()